# Create a simple tensor with random items

In [1]:
import numpy as np

# Suppress scientific notation
np.set_printoptions(suppress=True, precision=8)

# Generate randomly distributed parameters
params = np.random.uniform(low=-50, high=150, size=20)

# Make sure important values are at the beginning for better debugging
params[0] = params.max() + 1
params[1] = params.min() - 1
params[2] = 0

# Round each number to the second decimal place
params = np.round(params, 2)

# Print the parameters
print(params)

[135.5  -45.5    0.    88.83 130.06 134.5   71.41  60.69 111.94 -20.48
  33.76  27.16 107.45  70.34 -44.5  -17.51   3.    14.98 -21.28  48.07]


# Define the quantization methods and quantize

In [5]:
def clamp(params_q: np.array, lower_bound: int, upper_bound: int) -> np.array:
    params_q[params_q < lower_bound] = lower_bound
    params_q[params_q > upper_bound] = upper_bound
    return params_q


def asymmetric_quantization(params: np.array, bits: int) -> tuple[np.array, float, int]:
    # Calculate the scale and zero point
    alpha = np.max(params)
    beta = np.min(params)
    scale = (alpha - beta) / (2**bits - 1)
    zero = -1 * np.round(beta / scale)
    lower_bound, upper_bound = 0, 2**bits - 1
    # Quantize the parameters
    quantized = clamp(np.round(params / scale + zero), lower_bound, upper_bound).astype(
        np.int32
    )
    return quantized, scale, zero


def asymmetric_dequantize(params_q: np.array, scale: float, zero: int) -> np.array:
    return (params_q - zero) * scale


def symmetric_dequantize(params_q: np.array, scale: float) -> np.array:
    return params_q * scale


def symmetric_quantization(params: np.array, bits: int) -> tuple[np.array, float]:
    # Calculate the scale
    alpha = np.max(np.abs(params))
    scale = alpha / (2 ** (bits - 1) - 1)
    lower_bound = -(2 ** (bits - 1))
    upper_bound = 2 ** (bits - 1) - 1
    # Quantize the parameters
    quantized = clamp(np.round(params / scale), lower_bound, upper_bound).astype(
        np.int32
    )
    return quantized, scale


def quantization_error(params: np.array, params_q: np.array):
    # calculate the MSE
    return np.mean((params - params_q) ** 2)


(asymmetric_q, asymmetric_scale, asymmetric_zero) = asymmetric_quantization(params, 8)
(symmetric_q, symmetric_scale) = symmetric_quantization(params, 8)

print(f"Original:")
print(np.round(params, 2))
print("")
print(f"Asymmetric scale: {asymmetric_scale}, zero: {asymmetric_zero}")
print(asymmetric_q)
print("")
print(f"Symmetric scale: {symmetric_scale}")
print(symmetric_q)

Original:
[135.5  -45.5    0.    88.83 130.06 134.5   71.41  60.69 111.94 -20.48
  33.76  27.16 107.45  70.34 -44.5  -17.51   3.    14.98 -21.28  48.07]

Asymmetric scale: 0.7098039215686275, zero: 64.0
[255   0  64 189 247 253 165 150 222  35 112 102 215 163   1  39  68  85
  34 132]

Symmetric scale: 1.0669291338582678
[127 -43   0  83 122 126  67  57 105 -19  32  25 101  66 -42 -16   3  14
 -20  45]


In [6]:
# Dequantize the parameters back to 32 bits
params_deq_asymmetric = asymmetric_dequantize(
    asymmetric_q, asymmetric_scale, asymmetric_zero
)
params_deq_symmetric = symmetric_dequantize(symmetric_q, symmetric_scale)

print(f"Original:")
print(np.round(params, 2))
print("")
print(f"Dequantize Asymmetric:")
print(np.round(params_deq_asymmetric, 2))
print("")
print(f"Dequantize Symmetric:")
print(np.round(params_deq_symmetric, 2))

Original:
[135.5  -45.5    0.    88.83 130.06 134.5   71.41  60.69 111.94 -20.48
  33.76  27.16 107.45  70.34 -44.5  -17.51   3.    14.98 -21.28  48.07]

Dequantize Asymmetric:
[135.57 -45.43   0.    88.73 129.89 134.15  71.69  61.04 112.15 -20.58
  34.07  26.97 107.18  70.27 -44.72 -17.75   2.84  14.91 -21.29  48.27]

Dequantize Symmetric:
[135.5  -45.88   0.    88.56 130.17 134.43  71.48  60.81 112.03 -20.27
  34.14  26.67 107.76  70.42 -44.81 -17.07   3.2   14.94 -21.34  48.01]


In [7]:
# Calculate the quantization error
print(
    f'{"Asymmetric error: ":>20}{np.round(quantization_error(params, params_deq_asymmetric), 2)}'
)
print(
    f'{"Symmetric error: ":>20}{np.round(quantization_error(params, params_deq_symmetric), 2)}'
)

  Asymmetric error: 0.04
   Symmetric error: 0.06
